In [1]:
import cv2
import imutils
import numpy as np
import pytesseract
import matplotlib.pyplot as plt
import os
import csv
#_____------------------------_____-UPDATE TESSERACT OCR PATH OF REQIURED---------
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"
#-------------------------------------------------------------------
folder_path = r"C:\Users\asus\Downloads\LP\New Images"
#=----------------------------------------- UPDATE IMAGES FOLDER PATH--------------------
csv_file = open("output.csv", mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Image Files   ', '   License Plate'])
csv_writer.writerow(['---------------------------------------'])

for filename in os.listdir(folder_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(folder_path, filename)
        
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)
        #plt.imshow(img)
        #img = cv2.resize(img, (500, 500))
        #cv2.imshow("image filename",img)

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        gray = cv2.bilateralFilter(gray, 13, 15, 15)

        edged = cv2.Canny(gray, 30, 200) 

        contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(contours)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
        screenCnt = None

        for c in contours:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.018 * peri, True)
            if len(approx) == 4:
                screenCnt = approx
                break

        if screenCnt is None:
            detected = 0
            print("No contour detected")
        else:
            detected = 1

        if detected == 1:
            cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

        mask = np.zeros(gray.shape, np.uint8)
        new_image = cv2.drawContours(mask, [screenCnt], 0, 255, -1)
        new_image = cv2.bitwise_and(img, img, mask=mask)

        (x, y) = np.where(mask == 255)
        (topx, topy) = (np.min(x), np.min(y))
        (bottomx, bottomy) = (np.max(x), np.max(y))
        cropped = gray[topx:bottomx + 1, topy:bottomy + 1]

        
        license_plate_text = pytesseract.image_to_string(cropped, config=f'--psm 10 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
        csv_writer.writerow([filename,  " == "  ,license_plate_text])

csv_file.close()
